In [0]:
df=spark.read.table("dev.bronze.sales")

In [0]:
df_final=df.dropDuplicates().dropna().drop("ingestion_date")

In [0]:
df_final.write.mode("overwrite").saveAsTable("dev.silver.sales_cleaned")

In [0]:
%sql
DROP TABLE dev.silver.sales;

In [0]:
%sql
----select * from dev.silver.sales_cleaned

Sliver


In [0]:
df=spark.read.table("dev.bronze.sales") 

df_final=df.dropDuplicates().dropna().drop("ingestion_date") 

df_final.write.mode("overwrite").saveAsTable("dev.silver.sales_cleaned") 


In [0]:

%sql 

CREATE OR REPLACE TABLE dev.silver.products_scd (product_id int, product_name string, product_category string, product_price double) 

In [0]:
%sql 
WITH deduplicated_source AS ( 
SELECT
s.*, 
ROW_NUMBER() OVER (PARTITION BY product_id ORDER BY seqNum DESC) as row_num 
FROM dev.bronze.product s 
) 
MERGE INTO dev.silver.products_scd t 
USING (SELECT * FROM deduplicated_source WHERE row_num = 1 
) s 
ON t.product_id = s.product_id 
WHEN MATCHED and s.operation='UPDATE'  
THEN  
UPDATE SET  
t.product_name = s.product_name, 
t.product_category = s.product_category, 
t.product_price = s.product_price 
WHEN MATCHED AND s.operation='DELETE' 
THEN  
DELETE 
WHEN NOT MATCHED THEN  
INSERT  
(product_id, product_name, product_category, product_price)  
VALUES  
(s.product_id, s.product_name, s.product_category, s.product_price) 


In [0]:
%sql
select * from dev.silver.products_scd 

In [0]:
%sql
select * from dev.bronze.customers

In [0]:
%sql
CREATE OR REPLACE TABLE dev.silver.customers_scd (customer_id int, customer_name string, customer_email string, customer_city string, customer_state string, is_current BOOLEAN,effective_date date, end_date date, version_number int ) 

In [0]:
%sql

-- Step 1: Deduplicate source
WITH deduplicated_source AS (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY seqNum DESC) AS row_num
  FROM dev.bronze.customers
),

-- Step 2: Filter for latest records
latest_source AS (
  SELECT * FROM deduplicated_source
  WHERE row_num = 1
),

-- Step 3: Find records that need to be updated (i.e., values changed)
changed_records AS (
  SELECT s.*
  FROM latest_source s
  JOIN dev.silver.customers_scd t
    ON s.customer_id = t.customer_id
   AND t.is_current = true
  WHERE s.operation = 'UPDATE'
    AND (
         s.customer_name != t.customer_name OR
         s.customer_email != t.customer_email OR
         s.customer_address != t.customer_address
       )
)

-- Step 4: Apply SCD Type 2 logic
MERGE INTO dev.silver.customers_scd t
USING latest_source s
ON t.customer_id = s.customer_id AND t.is_current = true

-- Close out old record if changed
WHEN MATCHED AND s.operation = 'UPDATE'
     AND (
          s.customer_name != t.customer_name OR
          s.customer_email != t.customer_email OR
          s.customer_address != t.customer_address
     )
THEN UPDATE SET
  t.is_current = false,
  t.end_date = current_timestamp()

-- Mark deleted records as no longer current
WHEN MATCHED AND s.operation = 'DELETE'
THEN UPDATE SET
  t.is_current = false,
  t.end_date = current_timestamp()

-- Insert new version (either new or updated)
WHEN NOT MATCHED AND s.operation != 'DELETE' THEN
INSERT (
  customer_id,
  customer_name,
  customer_email,
  customer_address,
  is_current,
  start_date,
  end_date
)
VALUES (
  s.customer_id,
  s.customer_name,
  s.customer_email,
  s.customer_address,
  true,
  current_timestamp(),
  null
)
